### __Option Pricing on S&P 500 Daily Risk Control 10% Index__

__The S&P 500 Daily Risk Control 10% Index (SPXT10UE) is part of S&P Dow Jones Risk Control Indices familly and use the below parameters__

| ***Index Name*** | ***Underlying Risk Index*** | ***Risk Control Level*** | ***Maximum Leverage*** | ***Interest Rate*** | ***Volatility Calculation*** | ***Return Frequency for Volatility*** | ***Lag to Rebalancing Date*** | ***Decay Factor (Short-Term)*** | ***Decay Factor (Long-Term)*** | ***Rebalancing Frequency*** | ***Launch Date*** | ***Bloomberg Tickers***                                                           |
|------------------------------------------------|------------------------------------------------------|------------------------|----------------------|-------------------------|----------------------------|--------------------------------------|-----------------------------|-------------------------------|------------------------------|---------------------------|---------------|---------------------------------------------------------------------------------|
| S&P 500 Daily Risk Control 10% Index|S&P 500 Total Return: SPTR (USD) | 10% | 150%                 | SOFR + 0.02963*         | Exponentially weighted    | Daily                                | 2 days                      | 94%                           | 97%                          | Daily                     | 10-Sep-09     | ***Excess Return:*** SPXT5UE (USD)|


__The S&P 500 Daily Risk Control indices are computed using the below methodology__

$$
\text{Risk Control ER Index Value}_t 
= 
\text{RiskControlERIndexValue}_{rb}
\,\times\,
\Biggl[
1
\;+\;
K_{rb}\,\biggl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-1}} \;-\; 1\biggr)
\;-\;
K_{rb}\,\Bigl(
  \prod_{i=rb+1}^{t}\bigl(1 + \text{InterestRate}_{i-1} \times \frac{D_{i-1,i}}{360}\bigr)
  \;-\; 1
\Bigr)
\Biggr]
$$


$$
K_{rb} 
= 
\min\!\Bigl(\text{Max }K,\;\frac{\text{Target Volatility}}{\text{Realized Volatility}_{rb-d}}\Bigr)
$$


$$
\text{RealizedVolatility}_t 
  = \max\bigl(\text{RealizedVolatility}_{S,t}, \text{RealizedVolatility}_{L,t}\bigr)
$$

$$
\text{RealizedVolatility}_{S,t} 
  = \sqrt{\frac{252}{n}\,\text{Variance}_{S,t}}
$$

$$
\text{RealizedVolatility}_{L,t}
  = \sqrt{\frac{252}{n}\,\text{Variance}_{L,t}}
$$

$$
\text{Variance}_{S,t}
  = \lambda_S\,\text{Variance}_{S,t-1}
   + \bigl(1 - \lambda_S\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

$$
\text{Variance}_{L,t}
  = \lambda_L\,\text{Variance}_{L,t-1}
   + \bigl(1 - \lambda_L\bigr)\,\left[
       \ln\!\Bigl(\frac{\text{UnderlyingIndex}_t}{\text{UnderlyingIndex}_{t-n}}\Bigr)
     \right]^2
$$

In [3]:
# -------
# IMPORT
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from typing import Dict, Optional, Union

In [4]:
def simulate_hhw_paths_with_risk_control(
    
    # Heston-Hull-White model parameters
    S0: float,
    v0: float,
    r0: float,
    b0: float,
    kappa_v: float,
    theta_v: float,
    sigma_v: float,
    kappa_r: float,
    theta_r: float,
    sigma_r: float,
    rho_sv: float,
    rho_sr: float,
    rho_vr: float,
    T: float,
    N: int,
    M: int,
    seed: int,

    # Risk control parameters
    initial_realized_variance: float,
    target_volatility: float = 0.1,
    lambda_s: float = 0.94,
    lambda_l: float = 0.97,
    max_k: float = 1.5,
    lag: int = 2,
    initial_risk_control_index_value: float = 100.0,
    interest_rate_basis: float = 0.0002963,
    excess_return = True,

    # Optional override for the first Brownian increment at t=1
    spot_scenario: float = None) -> Dict[int, Dict[str, np.ndarray]]:
    
    """
    Simulate Monte Carlo paths under the Heston-Hull-White model with risk control,
    allowing an override of the first Brownian increment dW1 at t=1 while preserving
    the correlation structure with dW2 and dW3.

    If override_dW1 is provided, it will replace the random dW1 at t=1 for all paths.
    """
    dt = T / M

    # Build correlation matrix and its Cholesky factor
    corr = np.array([
        [1.0, rho_sv, rho_sr],
        [rho_sv, 1.0, rho_vr],
        [rho_sr, rho_vr, 1.0]
    ])
    L = np.linalg.cholesky(corr)

    # Allocate arrays
    S = np.zeros((M + 1, N))
    v = np.zeros((M + 1, N))
    r = np.zeros((M + 1, N))
    S[0, :] = S0
    v[0, :] = v0
    r[0, :] = r0

    rng = np.random.default_rng(seed)
    sqrt_dt = np.sqrt(dt)

    for t in range(1, M + 1):

        # Generate independent standard normals
        Z = rng.standard_normal((3, N))

        # Override dW1 at t=1 if requested
        if spot_scenario is not None and t == 1:
            # Broadcast scalar to array if needed
            override_dW1 = ((np.log(1+spot_scenario) - (r0 + b0 - 0.5 * v0) * dt) / np.sqrt(v0))
            if np.isscalar(override_dW1):
                dW1_arr = np.full(N, override_dW1)
            else:
                dW1_arr = np.asarray(override_dW1)
                if dW1_arr.shape != (N,):
                    raise ValueError("override_dW1 must be scalar or shape (N,)")
            # Compute corresponding Z1 values from the override:
            # dW1 = L[0,0] * Z1 * sqrt(dt)  => Z1 = dW1 / (L[0,0] * sqrt(dt))
            Z1_override = dW1_arr / (L[0, 0] * sqrt_dt)
            Z[0, :] = Z1_override

        # Compute correlated increments
        dW = L @ Z * sqrt_dt
        dW1, dW2, dW3 = dW[0], dW[1], dW[2]

        # Full-truncation Euler for variance
        v_prev = np.maximum(v[t-1, :], 0.0)
        v[t, :] = (
            v[t-1, :]
            + kappa_v * (theta_v - v_prev) * dt
            + sigma_v * np.sqrt(v_prev) * dW2
        )
        v[t, :] = np.maximum(v[t, :], 0.0)

        # Hull-White short rate
        r[t, :] = (
            r[t-1, :]
            + kappa_r * (theta_r - r[t-1, :]) * dt
            + sigma_r * dW3
        )

        # Asset price log-Euler
        S[t, :] = (
            S[t-1, :]
            * np.exp((r[t-1, :] + b0 - 0.5 * v_prev) * dt + np.sqrt(v_prev) * dW1)
        )

    # Process each path and compute risk control index
    results = {}
    rc_results = {}
    s_results = {}
    K_results = {}
    for i in range(N):
        # Extract path data
        S_path = S[:, i]
        v_path = v[:, i]
        r_path = r[:, i]
        
        # 1. Compute risky asset returns
        risky_asset_returns = np.zeros(M + 1)
        risky_asset_returns[1:] = (S_path[1:] - S_path[:-1]) / S_path[:-1]
        
        # 2. EWMA variances
        variance_s = np.zeros(M + 1)
        variance_l = np.zeros(M + 1)
        variance_s[0] = initial_realized_variance
        variance_l[0] = initial_realized_variance
        
        for j in range(1, M + 1):
            r2 = risky_asset_returns[j] ** 2 if not np.isnan(risky_asset_returns[j]) else 0.0
            # Short-run variance
            variance_s[j] = (1 - lambda_s) * r2 + lambda_s * variance_s[j - 1]
            # Long-run variance
            variance_l[j] = (1 - lambda_l) * r2 + lambda_l * variance_l[j - 1]
        
        # 3. Annualized volatilities
        volatility_s_annualized = np.sqrt(variance_s * 252)
        volatility_l_annualized = np.sqrt(variance_l * 252)
        
        # 4. Realized volatility (max of the two), lagged and back-filled
        realized_volatility = np.maximum(volatility_s_annualized, volatility_l_annualized)
        
        # Apply lag and back-fill (reproduce pandas shift(lag).bfill() behavior)
        if lag > 0:
            realized_volatility_lagged = np.zeros_like(realized_volatility)
            realized_volatility_lagged[lag:] = realized_volatility[:-lag]
            # Back-fill: fill first 'lag' positions with the first shifted value
            # This reproduces df.shift(lag).bfill() behavior
            realized_volatility_lagged[:lag] = realized_volatility[0]
            realized_volatility = realized_volatility_lagged
        
        # 5. Leverage factor K, capped at max_k
        K = np.minimum(target_volatility / realized_volatility, max_k)
        
        # 6. Interest rate (shifted by 1)
        interest_rate = np.zeros(M + 1)
        interest_rate[1:] = (r_path[:-1] + interest_rate_basis)
        
        # 7. Risk control index returns
        if excess_return:
            risk_control_index_returns = K * (risky_asset_returns - interest_rate / 252)
        else:
            risk_control_index_returns = K * (risky_asset_returns)
        
        # 8. Cumulative index level
        # Handle NaN values by setting them to 0
        risk_control_index_returns = np.where(np.isnan(risk_control_index_returns), 
                                            0.0, risk_control_index_returns)
        
        risk_control_index_value = np.zeros(M + 1)
        risk_control_index_value[0] = initial_risk_control_index_value
        
        for j in range(1, M + 1):
            risk_control_index_value[j] = (risk_control_index_value[j-1] * 
                                         (1 + risk_control_index_returns[j]))
        
        # Store results for this path
        results[i] = {
            'S': S_path,
            'v': v_path,
            'r': r_path,
            'risky_asset_returns': risky_asset_returns,
            'variance_s': variance_s,
            'variance_l': variance_l,
            'volatility_s_annualized': volatility_s_annualized,
            'volatility_l_annualized': volatility_l_annualized,
            'realized_volatility': realized_volatility,
            'K': K,
            'interest_rate': interest_rate,
            'risk_control_index_returns': risk_control_index_returns,
            'risk_control_index_value': risk_control_index_value
        }
        K_results[i] = K
        s_results[i] = S_path
        rc_results[i] = risk_control_index_value
        
    return K_results, s_results, rc_results, results
    


In [5]:
# Define parameters for the simulation ~ Black-Scholes
params = {

    # Heston-Hull-White model parameters
    'S0': 100.0, 
    'v0': 0.1**2, 
    'r0': 0.04, 
    'b0': 0.0002963,
    
    # Heston parameters
    'kappa_v': 10.0, 
    'theta_v': 0.1**2, 
    'sigma_v': 0.0001,
    'rho_sv': 0.0, 

    # Hull-White parameters
    'kappa_r': 1.0, 
    'theta_r': 0.04, 
    'sigma_r': 0.1,
    'rho_sr': 0.15, 

    # X parameter
    'rho_vr': 0.0,

    # Simulation parameters
    'T': 1.0, 
    'N': 20000, 
    'M': 252, 
    'seed': 42,

    # Risk control parameters
    'initial_realized_variance': 0.1**2 / 252,
    'excess_return' : False,

    # Scnario parameters
    'spot_scenario' : None,  # No override for the first Brownian increment
}

K_results, s_results, rc_results, results = simulate_hhw_paths_with_risk_control(**params)

df_K = pd.DataFrame(K_results)
df_s = pd.DataFrame(s_results)
df_rc = pd.DataFrame(rc_results)

forward = df_rc.iloc[-1].mean()
atm_call_price = (df_rc.iloc[-1]-100).clip(lower=0).mean()

print(f"Forward: {forward:.4f}, ATM Call Price: {atm_call_price:.4f}")

Forward: 104.2187, ATM Call Price: 7.0284


In [6]:
# Define parameters for the simulation ~ Black-Scholes
params = {

    # Heston-Hull-White model parameters
    'S0': 100.0, 
    'v0': 0.1**2, 
    'r0': 0.04, 
    'b0': 0.0002963,
    
    # Heston parameters
    'kappa_v': 10.0, 
    'theta_v': 0.1**2, 
    'sigma_v': 0.0001,
    'rho_sv': 0.0, 

    # Hull-White parameters
    'kappa_r': 1.0, 
    'theta_r': 0.04, 
    'sigma_r': 0.1,
    'rho_sr': -0.15, 

    # X parameter
    'rho_vr': 0.0,

    # Simulation parameters
    'T': 1.0, 
    'N': 20000, 
    'M': 252, 
    'seed': 42,

    # Risk control parameters
    'initial_realized_variance': 0.1**2 / 252,
    'excess_return' : False,

    # Scnario parameters
    'spot_scenario' : None,  # No override for the first Brownian increment
}

K_results, s_results, rc_results, results = simulate_hhw_paths_with_risk_control(**params)

df_K = pd.DataFrame(K_results)
df_s = pd.DataFrame(s_results)
df_rc = pd.DataFrame(rc_results)

forward = df_rc.iloc[-1].mean()
atm_call_price = (df_rc.iloc[-1]-100).clip(lower=0).mean()

print(f"Forward: {forward:.4f}, ATM Call Price: {atm_call_price:.4f}")

Forward: 104.1106, ATM Call Price: 6.5732


In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, clear_output

# Controls
S0 = widgets.FloatText(value=100.0, description='S0')
v0 = widgets.FloatText(value=0.1**2, description='v0 (var)')
r0 = widgets.FloatText(value=0.04, description='r0')
b0 = widgets.FloatText(value=0.0002963, description='b0')

kappa_v = widgets.FloatText(value=10.0, description='kappa_v')
theta_v = widgets.FloatText(value=0.1**2, description='theta_v (var)')
sigma_v = widgets.FloatText(value=0.0001, description='sigma_v')
rho_sv = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.01, description='rho_sv', continuous_update=False)

kappa_r = widgets.FloatText(value=1.0, description='kappa_r')
theta_r = widgets.FloatText(value=0.04, description='theta_r')
sigma_r = widgets.FloatText(value=0.1, description='sigma_r')
rho_sr = widgets.FloatSlider(value=-0.15, min=-1.0, max=1.0, step=0.01, description='rho_sr', continuous_update=False)

rho_vr = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.01, description='rho_vr', continuous_update=False)

T = widgets.FloatText(value=1.0, description='T (yrs)')
N = widgets.IntText(value=5000, description='N paths')
M = widgets.IntText(value=252, description='M steps')
seed = widgets.IntText(value=42, description='seed')

initial_realized_variance = widgets.FloatText(value=(0.1**2)/252, description='init var')
target_volatility = widgets.FloatText(value=0.10, description='target vol')
lambda_s = widgets.FloatSlider(value=0.94, min=0.80, max=0.999, step=0.001, description='lambda_s', continuous_update=False)
lambda_l = widgets.FloatSlider(value=0.97, min=0.80, max=0.999, step=0.001, description='lambda_l', continuous_update=False)
max_k = widgets.FloatText(value=1.5, description='max_k')
lag = widgets.IntSlider(value=2, min=0, max=10, step=1, description='lag', continuous_update=False)
initial_risk_control_index_value = widgets.FloatText(value=100.0, description='RC0')
interest_rate_basis = widgets.FloatText(value=0.0002963, description='rate basis')

excess_return = widgets.Checkbox(value=False, description='excess_return')
use_spot_scenario = widgets.Checkbox(value=False, description='use spot_scenario')
spot_scenario = widgets.FloatText(value=0.0, description='spot return')

run_button = widgets.Button(description='Run simulation', button_style='primary')
out = widgets.Output(layout=Layout(border='1px solid #ccc'))

# Layout
heston_box = VBox([
    HBox([S0, v0, r0, b0]),
    HBox([kappa_v, theta_v, sigma_v, rho_sv])
])

hw_box = VBox([
    HBox([kappa_r, theta_r, sigma_r, rho_sr]),
    HBox([rho_vr])
])

sim_box = VBox([
    HBox([T, N, M, seed])
])

rc_box = VBox([
    HBox([initial_realized_variance, target_volatility, max_k, lag]),
    HBox([lambda_s, lambda_l, initial_risk_control_index_value, interest_rate_basis]),
    HBox([excess_return, use_spot_scenario, spot_scenario])
])

ui = VBox([
    widgets.HTML('<b>Heston-Hull-White parameters</b>'),
    heston_box,
    widgets.HTML('<b>Hull-White and correlations</b>'),
    hw_box,
    widgets.HTML('<b>Simulation</b>'),
    sim_box,
    widgets.HTML('<b>Risk Control</b>'),
    rc_box,
    run_button,
    out
])

def run_callback(_):
    with out:
        clear_output(wait=True)
        params = {
            'S0': S0.value,
            'v0': v0.value,
            'r0': r0.value,
            'b0': b0.value,
            'kappa_v': kappa_v.value,
            'theta_v': theta_v.value,
            'sigma_v': sigma_v.value,
            'kappa_r': kappa_r.value,
            'theta_r': theta_r.value,
            'sigma_r': sigma_r.value,
            'rho_sv': float(rho_sv.value),
            'rho_sr': float(rho_sr.value),
            'rho_vr': float(rho_vr.value),
            'T': T.value,
            'N': int(N.value),
            'M': int(M.value),
            'seed': int(seed.value),
            'initial_realized_variance': initial_realized_variance.value,
            'target_volatility': target_volatility.value,
            'lambda_s': float(lambda_s.value),
            'lambda_l': float(lambda_l.value),
            'max_k': max_k.value,
            'lag': int(lag.value),
            'initial_risk_control_index_value': initial_risk_control_index_value.value,
            'interest_rate_basis': interest_rate_basis.value,
            'excess_return': bool(excess_return.value),
            'spot_scenario': (spot_scenario.value if use_spot_scenario.value else None),
        }
        K_results, s_results, rc_results, _ = simulate_hhw_paths_with_risk_control(**params)
        df_rc = pd.DataFrame(rc_results)
        forward = df_rc.iloc[-1].mean()
        atm_call_price = (df_rc.iloc[-1] - 100).clip(lower=0).mean()
        print(f"Forward: {forward:.4f}, ATM Call Price: {atm_call_price:.4f}")

run_button.on_click(run_callback)

display(ui)
